1. 대학교를 작성한 모든 리스트 앞에는 '학력'이 있음
2. 단과 -> '대학' / 대학교 -> 00대학교

In [ ]:
import pandas as pd
import numpy as np
import re
import csv

data = pd.read_csv('rocket.csv')

def strip(series):
    series = series.str.strip('\r\n\s ')
    return series


def split(series):
    series = series.str.split('\r?\n,?\s+')  # 공백문자 제거
    return series


def basic(data):
    data = data.drop(['Unnamed: 0', 'name'], axis=1)
    data = data.drop_duplicates()
    data = data[data.isna().sum(axis=1) < data.shape[1]]
    data = data.fillna('')
    data = data.reset_index(drop=True)
    data['id'] = data.index
    return data

In [ ]:
#school 데이터에서 연도 뽑기 / 나머지 '' 처리
data = basic(data)
school = data['school']
school = school.str.replace('학력','')
school = strip(school)
school = school.str.split('(\n +){3,}')

In [ ]:
# 공백원소 제거
for i in range(0,44227):
    school[i] = list(filter(str.strip, school[i]))
# 졸업, 휴학, 중퇴, 재학이 있는 학력만 남김
for i in range(0,44227):
    school[i] = [j for j in school[i] if (j.find("졸업"))*(j.find("휴학"))*(j.find("중퇴"))*(j.find("재학")) < 0]
# 고등학교 학력 제거
for i in range(0,44227):
    school[i] = [j for j in school[i] if j.find("고등") == -1]
#프로젝트명에 "졸업"이 자주 들어감, 프로젝트 들어간 부분 제거
for i in range(0,44227):
    school[i] = [j for j in school[i] if j.find("젝트") == -1]


In [ ]:
notnull =[]
#null 제거, 학력이 4개 이상인 47명 제외함
for i in range(0,44227):
    if((len(school[i]) > 0) & (len(school[i]) < 4)):
        notnull.append(i)  
len(notnull)

In [ ]:
two = []
for i in range(0,44227):
    if ((len(school[i])) == 2):
       two.append(i)  
thr = []
for i in range(0,44227):
    if ((len(school[i])) == 3):
       thr.append(i)  

In [ ]:
df = pd.DataFrame(school[notnull])
df1 = pd.DataFrame(df['school'].values.tolist(), columns = ['학력1','학력2','학력3'])

#학력1
df['학력1'] = df1['학력1'].tolist()
df['학력1'] = df['학력1'].str.split("\r?\n")
df2 = pd.DataFrame(df['학력1'].values.tolist())
df['학교1'] = df2[0].tolist()
df['전공1'] = df2[1].tolist()
df['재학기간1'] = df2[2].tolist()
df = df.drop(df.columns[0:2], axis='columns')

#학력2
df['학력2'] = df1['학력2'].tolist()
df['학력2'] = df['학력2'].str.split("\r?\n")

df3 = pd.DataFrame(data = df['학력2'][two].values.tolist(),index = two )
df3 = df3.drop(df3.columns[3:], axis='columns')

df3 = df3.rename(columns={df3.columns[0]:'학교2', df3.columns[1]: '전공2', df3.columns[2]:'재학기간2'})
df5 = df.join(df3, how='left')

#학력3
df['학력3'] = df1['학력3'].tolist()
df['학력3'] = df['학력3'].str.split("\r?\n")

df4 = pd.DataFrame(data = df['학력3'][thr].values.tolist(),index = thr )
df4 = df4.drop(df4.columns[3:], axis='columns')
df4 = df4.rename(columns={df4.columns[0]:'학교3', df4.columns[1]: '전공3', df4.columns[2]:'재학기간3'})

dfinal = df5.join(df4, how = 'left')
dfinal = dfinal.drop(dfinal.columns[3], axis='columns')
dfinal

dfinal['재학기간1-1']=dfinal['재학기간1'].str[8:13]

dfinal['재학기간1-2']=dfinal['재학기간1'].str[16:21]

dfinal['재학여부1']=dfinal['재학기간1'].str[22:26]

dfinal['재학기간1-2'] = dfinal['재학기간1-2'].replace('[\-]', '', regex=True)


dfinal['재학기간2-1']=dfinal['재학기간2'].str[8:13]

dfinal['재학기간2-2']=dfinal['재학기간2'].str[16:21]

dfinal['재학여부2']=dfinal['재학기간2'].str[22:26]

dfinal['재학기간2-2'] = dfinal['재학기간2-2'].replace('[\-]', '', regex=True)


dfinal['재학기간3-1']= dfinal['재학기간3'].str[8:13]

dfinal['재학기간3-2']= dfinal['재학기간3'].str[16:21]

dfinal['재학여부3']=dfinal['재학기간3'].str[22:26]

dfinal['재학기간3-2'] = dfinal['재학기간3-2'].replace('[\-]', '', regex=True)

split_df = dfinal['전공1'].str.split('(\ +)', expand=True)
dfinal['전공1-학과']= split_df[2]
dfinal['전공1-전공(학사)']= split_df[4]

split_df = dfinal['전공2'].str.split('(\ +)', expand=True)
dfinal['전공2-학과']= split_df[2]
dfinal['전공2-전공(학사)']= split_df[4]

split_df = dfinal['전공3'].str.split('(\ +)', expand=True)
dfinal['전공3-학과']= split_df[2]
dfinal['전공3-전공(학사)']= split_df[4]



dfinal = dfinal.drop(dfinal.columns[1:3], axis='columns')
dfinal = dfinal.drop(dfinal.columns[2:4], axis='columns')
dfinal = dfinal.drop(dfinal.columns[3:5], axis='columns')
dfinal